## This notebook prepares testing / training data for the supervised learning of LCS

*The basic goal is to train a neural network to label RCLVs based on the LAVD field using the existing algorithm as a training dataset.*

#### Our inputs will be images containing:
- LAVD field
- Labeled binary mask of eddies detected using RCLV

The LAVD field was generated following the QG model setup in [Zhang et al. (2020)](https://doi.org/10.3390/fluids5010002), which can be reproduced in **02_RCLV_Zhang2020.ipynb**. Eddies are identified using the RCLV method and implemented with `floater` using a coherency index of -0.75 and minimum area of 200 pixels.  

#### Some questions:

How many images do we need to be statistically independent? 100? 1000?

Are we going to use augmentation (rotation, translation, flipping, etc.)?

In [1]:
import xarray as xr
from tensorflow import keras


In [2]:
# Import zarr files created from 02_RCLV_Zhang2020.ipynb
LAVD = xr.open_zarr()

labels = xr.open_zarr()

In [ ]:
keras.backend.set_image_data_format('channels_last')

model_size = 512
nchannels = 1

input_layer = keras.Input(shape=(model_size, model_size, nchannels))
conv_layer0 = keras.layers.Conv2D(10, 2, activation='relu')(input_layer)
conv_layer1 = keras.layers.Conv2D(10, 2, activation='relu')(conv_layer0)
conv_layer2 = keras.layers.Conv2D(10, 2, activation='relu')(conv_layer1)
sum_layer = keras.layers.Conv2D(1, 1, activation='softmax')(conv_layer2)

# todo: add "residual blocks", skip connections

model = keras.Model(inputs=input_layer, outputs=sum_layer)

optimizer = keras.optimizers.Adam()

# probably want to use cross entropy for loss
model.compile(loss='mse', optimizer=optimizer)
model.summary()